![](problem_statements/E026.png)

# Input

In [183]:
T = int(input().strip())

2


In [184]:
N_list = []
for _ in range(T):
    N_list.append(int(input().strip()))

5
10


# Steps
Get the recurring part of a decimal $1/d$

In [185]:
def get_recurrence_length(d):
    "Function to get the decimal expansion of 1/d"
    decimal = ''
        
    current_remainder = 10
    remainder_list = []

    while current_remainder!=0 and current_remainder not in remainder_list:
        #Division quotient goes into the decimal...
        decimal = decimal + str(current_remainder//d)

        #Update the remainder list...
        remainder_list.append(current_remainder)

        #Update the remainder for next iteration...
        current_remainder = (current_remainder % d)*10

    #Calculate recurrence length
    if current_remainder==0:
        recurrence_length = 0
    else:
        recurrence_length = len(remainder_list) - remainder_list.index(current_remainder)
        decimal = decimal[:-recurrence_length] + '(' + decimal[-recurrence_length:] + ')'
    return recurrence_length #,'0.' + decimal

# Unoptimized Solution
Now, write a definition with input *num* to run the above for all $d<num$, and find the longest recurring decimal

# Optimized Solution
We take advantage of the following theorems of recurring decimals:
https://en.wikipedia.org/wiki/Repeating_decimal#Other_properties_of_repetend_lengths

Specifically:
* If $p$ is prime, the period of $\frac{1}{p}$ divides evenly into $p-1$
* If $p, p', p''\ldots$ are distinct primes, the period of $\frac{1}{pp'p''}$ is the LCM of the periods of $\frac{1}{p}$,$\frac{1}{p'}$,$\frac{1}{p''}$ $\ldots$
* If $k$ and $k'$ have no common prime factors other than 2 and 5, then period of $\frac{1}{kk'}$ is the LCM of the periods of $\frac{1}{k}$, $\frac{1}{k'}$
* For some prime $p$ if

$\qquad \text{period} \left( \frac{1}{p} \right) = \text{period} \left( \frac{1}{p^2} \right) = \ldots \text{period} \left( \frac{1}{p^m} \right)$

$\quad \ $ for some $m$, but

$\qquad \text{period} \left( \frac{1}{p^m} \right) \neq \text{period} \left( \frac{1}{p^{m+1}} \right)$

$\quad \ $ then for $c\geq 0$

$\qquad \text{period} \left( \frac{1}{p^{m+c}} \right) = p^c \cdot \text{period} \left( \frac{1}{p^m} \right)$

## Steps
1. Get the required primes, and their recurrence length...

In [186]:
def get_primes(num):
    "Return a list of all primes <= N"
    
    #Start with the initialization of all numbers from 1 to num as prime=True
    prime = [True if i>=2 else False for i in range(num + 1)]
    
    #Now, run a index from 2 to sqrt(num)
    i = 2
    while i*i<=num:
        #If i is prime, all multiples of i which are <=num will be composite...
        if prime[i]:
            for j in range(i*2,num+1,i):
                prime[j]=False
        i+=1
    #Now, list all the primes - i.e. all n such that prime[n]==True
    ans = []
    for i in range(num+1):
        if prime[i]:
            ans.append(i)
    return ans

In [187]:
MAX_N = 10000
primes_list = get_primes(MAX_N)

Now, calculate the recurrence lengths for all the primes in the list... We know that it's a divisor of $p-1$. So we can calculate all the $p-1$ decimals, and then look for repeating patterns in them

In [144]:
#___TO GET ALL FACTORS OF A NUMBER_____
from collections import Counter
def get_prime_factors(num):
    ans = Counter()
    n = num
    for p in primes_list: #Note - here we assume that primes_list is the complete list of primes <=sqrt(num)
        if p>int(num**0.5):
            break
        while n%p==0:
            ans[p] += 1
            n//=p
    #If there's a remaining factor...
    if n>1:
        ans[n] += 1
    return ans

from itertools import product
def get_all_factors(num):
    "List all the factors of num"
    prime_factors = get_prime_factors(num)
    exp_list = product(*[tuple(range(v+1)) for v in prime_factors.values()])
    ans = []
    for exp in exp_list:
        prod=1
        for b,e in zip(prime_factors.keys(),exp):
            prod *= b**e
        ans.append(prod)
    return sorted(ans)

In [145]:
def check_recurrence_length(string,l):
    "To check whether a string of integer multiple length if l has a recurrence of length l"
    #First, if the string doesn't divide into l equal parts, return False
    try:
        assert len(string)%l==0
    except AssertionError:
        return False
    #The first l digits...
    base = string[:l]
    #The remaining sets of l digits...
    for i in range(1,len(string)//l):
        if base!=string[i*l:(i+1)*l]:
            return False
    else:
        return True

In [146]:
def prime_recurrence_length(p):
    chk_str = str(10**(p-1)//p).rjust(p-1,'0')
    possible_recurrence_lengths = get_all_factors(p-1)
    for l in possible_recurrence_lengths:
        if check_recurrence_length(chk_str,l):
            break
    return l

Now, calculate prime recurrence length for all primes except $2,5$

In [149]:
ans_dict = {p:0 if (p==2 or p==5) else prime_recurrence_length(p) for p in primes_list}

Now, for the powers of these primes...

In [150]:
for p in primes_list:
    i = 1
    flag = True #a flag to check if the value c has been reached...
    while p**(i+1)<MAX_N:
        if p==2 or p==5:
            ans_dict[p**(i+1)] = 0
        elif flag and get_recurrence_length(p**(i+1))==ans_dict[p**i]:
            ans_dict[p**(i+1)] = ans_dict[p**i]
        else:
            flag=False
            ans_dict[p**(i+1)] = p*ans_dict[p**i]
        i += 1

Now, for the composite numbers...

In [171]:
def LCM(*args):
    "Function to find the LCM of all the args"
    try:
        exp = get_prime_factors(args[0])
    except IndexError:
        return 0
    for a in args[1:]:
        exp |= get_prime_factors(a)
    ans = 1
    for e in exp:
        ans *= e**exp[e]
    return ans

In [172]:
def composite_recurrence_length(c):
    "Recurrence length for composite numbers..."
    prime_factors = get_prime_factors(c)
    #Remove factors of 2 & 5
    try:
        _ = prime_factors.pop(2)
        _ = prime_factors.pop(5)
    except KeyError:
        pass
    parts = [ans_dict[k**prime_factors[k]] for k in prime_factors]
    return LCM(*parts)

Now, fill up ans_dict for all composites...

In [173]:
for num in range(4,10000):
    try:
        _ = ans_dict[num]
    except KeyError:
        ans_dict[num] = composite_recurrence_length(num)

In [179]:
#Put ans in the form of a list, where recurrene length of d is ans_list[d]
ans_list = [0,0] + [ans_dict[k] for k in range(2,10000)]

# Solution

In [180]:
for N in N_list:
    print(ans_list.index(max(ans_list[:N])))

3
7
9967
